 В этом задании необходимо разработать приложение для учёта товаров в
магазине. Приложение должно позволять создавать, обновлять и отображать
информацию о товарах. В процессе реализации вы будете применять
следующие инструментыZ
H typing для строгой типизации данных=
H dataclasses для упрощённого создания классов-сущностей товаров=
H ABC для определения интерфейса и обеспечения наследования=
H декораторы для логирования вызовов ключевых функций=
H logging для ведения системного логирования работы приложения.
Требования¼
áÒ Типизация (typing)Z
H Добавьте аннотации типов для всех функций и методовÒ
²Ò DataclassesZ
H Создайте класс товара с помощью @dataclass, содержащий поля:
идентификатор, название, цена и количествоÒ
ÝÒ Абстрактные классы (ABC)Z
H Определите абстрактный базовый класс с методами для обновления
информации о товаре и расчёта цены со скидкойÒ
H Создайте один или несколько классов, наследующих от этого базового
класса, с полной реализацией методовÒ
ÓÒ ДекораторыZ
H Напишите декоратор для логирования вызовов функций/методов (имя
функции, время вызова, аргументы). Примените его к ключевым
методамÒ
¡Ò Модуль loggingZ
H Настройте logging для записи важных событий (создание товара,
обновление, расчёт скидки) в приложение.

In [9]:
from typing import List, Dict, Union
from dataclasses import dataclass
from abc import ABC, abstractmethod
from datetime import datetime
import logging
from functools import wraps


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Обработчик файла с UTF-8
file_handler = logging.FileHandler('store.log', encoding='utf-8')  

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)


def log_function_call(func):
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        call_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        logger.info(f"Вызов функции: {func.__name__}, время: {call_time}, аргументы: {args}, {kwargs}")
        result = func(*args, **kwargs)
        return result
    return wrapper

@dataclass
class Product:
    id: int
    name: str
    price: float
    quantity: int


class ProductInterface(ABC):
    
    @abstractmethod
    def update(self, **kwargs) -> None:
        pass

    @abstractmethod
    def apply_discount(self, discount: float) -> float:
       
        pass


class StoreProduct(ProductInterface):
    def __init__(self, product: Product) -> None:
        self.product = product

    @log_function_call
    def update(self, **kwargs) -> None:
     
        for key, value in kwargs.items():
            if hasattr(self.product, key):
                setattr(self.product, key, value)
                logger.info(f"Обновлено: {key} -> {value}")

    @log_function_call
    def apply_discount(self, discount: float) -> float:
        new_price = self.product.price * (1 - discount / 100)
        self.product.price = round(new_price, 2)
        logger.info(f"Скидка {discount}% применена. Новая цена: {self.product.price:.2f}")
        return self.product.price


class Store:
    def __init__(self) -> None:
        self.products: Dict[int, StoreProduct] = {}

    @log_function_call
    def add_product(self, product: Product) -> None:
        self.products[product.id] = StoreProduct(product)
        logger.info(f"Товар {product.name} добавлен в магазин")

    @log_function_call
    def get_product(self, product_id: int) -> Union[StoreProduct, None]:
        
        return self.products.get(product_id)

    @log_function_call
    def list_products(self) -> List[Product]:
      
        return [p.product for p in self.products.values()]


def main():
   
    store = Store()

    store.add_product(Product(id=1, name="Телефон", price=1000.0, quantity=10))
    store.add_product(Product(id=2, name="Ноутбук", price=1500.0, quantity=5))

    product = store.get_product(1)
    if product:
        product.update(price=900.0, quantity=12)

    if product:
        product.apply_discount(10)
    products = store.list_products()
    for p in products:
        print(f"ID: {p.id}, Название: {p.name}, Цена: {p.price:.2f}, Количество: {p.quantity}")


if __name__ == "__main__":
    main()

ID: 1, Название: Телефон, Цена: 810.00, Количество: 12
ID: 2, Название: Ноутбук, Цена: 1500.00, Количество: 5
